# Projeto de Telefonia Móvel
### Este projeto tem por objetivo extrair os dados da fatura detalhada da operadora e transformar os dados em informações importantes e insights sobre a conta

##### Importando as bibliotecas necessárias

In [1]:
import pdfplumber
import re
import pandas as pd
from pathlib import Path
import numpy as np

#### Os dados foram importados do pdf com a biblioteca pdflumber, depois transformados em txt com o auxílio do Path


In [ ]:


pdf_url='/home/paula/Downloads/vivotest.pdf'
with pdfplumber.open(pdf_url) as pdf:
    
    first_page = pdf.pages[0]
    pages = pdf.pages
    
    with Path('/home/paula/Downloads/exemplo.txt').open(mode='w') as output_file:
        text=''
        for i,pg in enumerate(pages):
            text+= pages[i].extract_text()
        output_file.write(text)
    

#### Depois de extraídos, os dados precisavam ser padronizados. Com o auxílio da biblioteca RE, foi criado uma expressão regular para localizar os textos que continham formatos diferentes, logo depois foram removidas informações irrelevantes do plano para este estudo através do método replace.

In [3]:
busca='SMART EMPRESAS'
dadosFatura=''
dadosGerais=''
texto=''
with open('/home/paula/Downloads/exemplo.txt', 'r', encoding = 'utf-8') as arquivo:
        
    for linha in arquivo: 
        strug=' TE'
        strug1='MAS'
        if busca in linha:
                if (re.search(r'\s[0-9]*\s',linha)):
                    if strug in linha:
                        texto= linha.replace(' TE', '')
                    elif strug1 in linha:
                        texto= linha.replace(' MAS', '')
                    else:
                        texto= linha
                    dadosGerais+=texto
                elif strug in linha:
                    texto= linha.replace(' TE', '')
                    dadosFatura+=texto
                
                elif strug1 in linha:
                    texto= linha.replace(' MAS', '')
                    dadosFatura+=texto
                    
                else:
                    dadosFatura+=linha
                
                
                with Path('/home/paula/Downloads/dadosGeraisPlanos.txt').open(mode='w') as output_file:
                    output_file.write(dadosGerais)
               
                with Path('/home/paula/Downloads/dadosFatura.txt').open(mode='w') as output_file:
                    output_file.write(dadosFatura)

#### Os dados depois de tratados foram armazenados em arquivos diferentes para fazer a composição de dois dataframes bases relacionados à fatura.

In [ ]:
dfFatura = pd.read_csv('/home/paula/Downloads/dadosFatura.txt', sep="\n", header=None, names=["coluna"]) #names=["number", "tipo", "c"] 
divisao=dfFatura["coluna"].str.split(" ")

In [ ]:
numero=divisao.str.get(0)
desc1=divisao.str.get(1)
desc2=divisao.str.get(2)
plan=divisao.str.get(3)
period1=divisao.str.get(4)
period2=divisao.str.get(5)
period3=divisao.str.get(6)
price=divisao.str.get(7)

In [ ]:
dfFatura['Numero']=numero
dfFatura['desc1']=desc1
dfFatura['desc2']=desc2
dfFatura['internet']=plan
dfFatura['Descrição'] = dfFatura['desc1'] + " " + dfFatura['desc2'] + " " + dfFatura['internet']
dfFatura['Inicio']=period1
dfFatura['a']=period2
dfFatura['Final']=period3
dfFatura['Valor']=price.str.replace(',', '.') #convertendo pontuação
dfFatura['Valor']=dfFatura['Valor'].astype('float64')

In [ ]:
dfFatura.drop(['coluna'],axis=1,inplace=True)
dfFatura.drop(['a'],axis=1,inplace=True)
dfFatura.drop(['desc1'],axis=1,inplace=True)
dfFatura.drop(['desc2'],axis=1,inplace=True)
dfFatura.drop(['internet'],axis=1,inplace=True)
dfFatura.head()

In [ ]:
dfPlan = pd.read_csv('/home/paula/Downloads/dadosGeraisPlanos.txt', sep="\n", header=None, names=["dados"]) #names=["number", "tipo", "c"]
divisao=dfPlan["dados"].str.split(" ") 

In [ ]:
desc=divisao.str.get(0)
desc0=divisao.str.get(1)
internet=divisao.str.get(2)
quantidade=divisao.str.get(3)

In [ ]:
dfPlan['desc1']=desc
dfPlan['desc2']=desc0
dfPlan['Descrição'] = dfPlan['desc1'] + " " + dfPlan['desc2']
dfPlan['Internet']=internet
dfPlan['Quantidade']=quantidade
dfPlan['Quantidade']=dfPlan['Quantidade'].astype('int32')

In [ ]:
dfPlan.drop(['dados'],axis=1,inplace=True)
dfPlan.drop(['desc1'],axis=1,inplace=True)
dfPlan.drop(['desc2'],axis=1,inplace=True)
dfPlan.head()